In [ ]:
import pandas as pd
import pickle
from os import listdir
from os.path import isfile, join
import keras
import tensorflow as tf
import keras.backend as K

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [ ]:
mypath = "./LSTM_results/"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
files

In [ ]:
all_results_list = list()

for file in files:
    results_p = open(mypath + file,"rb")
    results_lst = pickle.load(results_p)
    all_results_list = all_results_list + results_lst

In [ ]:
all_results_list

In [ ]:
all_results_dict = dict()

In [ ]:
for item in all_results_list:
    for k, v in item.items():
        model_params = k.split("_")
        all_results_dict[("_".join(model_params[0:2]))] = {
            model_params[3].split(":")[0]: int(model_params[3].split(":")[1]),
            model_params[4].split(":")[0]: int(model_params[4].split(":")[1]),
            model_params[5].split(":")[0]: int(model_params[5].split(":")[1]),
            model_params[6].split(":")[0]: int(model_params[6].split(":")[1]),
            "val_loss": float(v["model-history"]["val_loss"][0]),
            "loss": float(v["model-history"]["loss"][0]),
            "MAE-holdout-set": float(v["MAE-holdout-set"].split()[1]),
            "app-bias-mean-holdout-set": float(v["app-bias-mean-holdout-set"])
        }

In [ ]:
all_results_df = pd.DataFrame(all_results_dict).T

In [ ]:
all_results_df.sort_values(by=["app-bias-mean-holdout-set"])

In [ ]:
def get_model(lst):
    for item in lst:
        for k, v in item.items():
            model_params = k.split("_")
            date_stamp = "_".join(model_params[0:2])
            if date_stamp == "model2019-09-22_11:54:01.286223":
                return v['model-object']

final_model = get_model(all_results_list)
print(K.eval(final_model.optimizer.lr))

for layer in final_model.layers:
    weights = layer.get_weights()
    print(weights)

In [ ]:
with open('./Models/LSTM_model_results.pkl', 'wb') as f:
    pickle.dump(all_results_df, f)

with open('./Models/LSTM_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)